# Triplet CNN Batch-All/Batch-Semi-Hard

In [ ]:
%autosave 0

In [ ]:
# Python Imaging Library
from PIL import Image    
import PIL.ImageOps

# Packages
import torch
from torch.utils.data import DataLoader,Dataset
from torch import optim     # 包含optimization algorithms
import torch.nn.functional as F     # 包含activation functions
from torch.autograd import Variable      # 以Variable形式嵌套激励函数 
import torch.nn as nn

# Torchvision 包含目前流行的数据集，模型结构和常用的图片转换工具等
import torchvision
import torchvision.datasets as dset    # 包含一些数据集
import torchvision.transforms as transforms  # 可对PIL.Image, Tensor进行变换
import torchvision.utils

# Others
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import linecache
import seaborn as sns

In [ ]:
class Config():
    training_dir = "./Datasets/att/training/"
    validation_dir = "./Datasets/att/validation"
    testing_dir = "./Datasets/att/testing/"
    batch_apn_path = './batch_apn_forall.txt'
    batch_all_hard_path = 'batch_all_hard.txt'
    train_batch_size = 32           #批样本数
    train_number_epoch = 400     #整批训练次数，即遍历了多少次所有的训练样本

In [ ]:
def imshow(img, text=None, should_save=False):     #数据集出图
    npimg = img.numpy()   #转numpy
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic', fontweight='bold',
                bbox={'facecolor':'white','alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
        
def show_plot(iteration, loss):                    #观察迭代损失
    plt.plot(iteration, loss)
    plt.xlabel('Iteration Number')
    plt.ylabel('Loss')
    plt.show

## generate mini-batch

In [ ]:
def MiniBatchAllTriplets(data_path, batch_path, num_class, num_pic):
    name_list = os.listdir(data_path)
    batch_name = random.sample(name_list, num_class)
    batch_name.sort()
    print(batch_name)
    batch_apn = []
    for k in range(len(batch_name)):
        name = batch_name[k]
        path1 = data_path + name + '/'
        pic_list1 = os.listdir(path1)
        batch_pic1 = random.sample(pic_list1, num_pic)
        batch_pic1.sort()                       #sort pics in the batch
        for i in range(len(batch_pic1)):
            anchor_pic = batch_pic1[i]
            anchor_pic = path1 + anchor_pic
            for j in range(len(batch_pic1)):
                if i!= j:
                    positive_pic = batch_pic1[j]
                    positive_pic = path1 + positive_pic
                    pic_ap = anchor_pic + '\t' + positive_pic    #make ap pairs per pic
                    for v in range(len(batch_name)):
                        name_other = batch_name[v]
                        path2 = data_path + name_other + '/'
                        pic_list2 = os.listdir(path2)
                        batch_pic2 = random.sample(pic_list2, num_pic)
                        batch_pic2.sort()
                        if v != k:
                            for q in range(len(batch_pic2)):
                                negative_pic = batch_pic2[q]
                                negative_pic = path2 + negative_pic
                                pic_apn = pic_ap + '\t' + negative_pic
                                batch_apn.append(pic_apn)
    fileObject = open(Config.batch_apn_path, 'w')  
    for pic_apn in batch_apn:  
        fileObject.write(pic_apn)  
        fileObject.write('\n')  
    fileObject.close()


In [ ]:
class_batch_size = 4
pic_batch_size = 5

MiniBatchAllTriplets(data_path = Config.training_dir, 
                     batch_path = Config.batch_apn_path,
                     num_class = class_batch_size,
                     num_pic = pic_batch_size)

## Loaders

In [ ]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('L')

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

## Load Triplets

In [ ]:
class GetBatchTriplet(Dataset):
    def __init__(self, img_path, txt_path, data_transforms=None, loader = default_loader):
        with open(txt_path) as input_file1:
            lines = input_file1.readlines()
            self.img_anchor = [os.path.join(img_path, line.strip().split('\t')[0]) for line in lines]
            self.img_positi = [os.path.join(img_path, line.strip().split('\t')[1]) for line in lines]   
            self.img_negati = [os.path.join(img_path, line.strip().split('\t')[2]) for line in lines]   
        self.data_transforms = data_transforms
        self.loader = loader

    def __len__(self):
        return len(self.img_anchor)

    def __getitem__(self, item):
        img_anchor = self.img_anchor[item]
        img_positi = self.img_positi[item]
        img_negati = self.img_negati[item]
        img_a = self.loader(img_anchor)
        img_p = self.loader(img_positi)
        img_n = self.loader(img_negati)
        
        if self.data_transforms is not None:
            try:
                img_a = self.data_transforms(img_a)
            except:
                print("Cannot transform image: {}".format(img_a))
            try:
                img_p = self.data_transforms(img_p)
            except:
                print("Cannot transform image: {}".format(img_s))
            try:
                img_n = self.data_transforms(img_n)
            except:
                print("Cannot transform image: {}".format(img_h))    
                
        return img_a, img_p, img_n

## Triplet CNN

In [ ]:
class TripletNetwork(nn.Module):                 # torch.nn.Module:所有神经网络模块的基类
    def __init__(self):
        super(TripletNetwork, self).__init__()
        self.cnn1 = nn.Sequential(               # torch.nn.Sequential:模块将按照在构造器中的顺序传递
            nn.ReflectionPad2d(1),               # 使用输入边界的反射来填充输入张量
            nn.Conv2d(                           # 对信号（由若干输入平面组成）进行2维卷积
                in_channels=1,                   # input height. 灰度图1层，RGB为3层，即图像的通道数
                out_channels=4,                  # n_filters 过滤器数目，过滤器提取的特征数
                kernel_size=3,                   # 卷积核 filter size 3x3
                stride=1,                        # filter movement/step 跳度
                padding=0,                       # 零填充。若想要con2d出来的图片长宽没有变化，当stride=1,padding=(kernel_size-1)/2
            ),
            nn.ReLU(inplace=True),               # inplace不是很懂
            nn.BatchNorm2d(4),                   # 对4维输入应用批标准化（一个小批量的2维输入和额外的通道尺寸）
            nn.Dropout2d(p=.2),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(
                in_channels=4,
                out_channels=8,
                kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),
            nn.Dropout2d(p=.2),
            
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(8, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),
            nn.Dropout2d(p=.2),
        )
        
        self.fc1 = nn.Sequential(                   # fc full connected
            nn.Linear(in_features=8*100*100,        # 线性层,y=Ax+b
                      out_features=500),
            nn.ReLU(inplace=True),
            
            nn.Linear(500,500),
            nn.ReLU(inplace=True),
            
            nn.Linear(500,50)
    )
    
    def forward_once(self,x):
        output = self.cnn1(x)
        output = output.view(output.size()[0],-1)   #Morvan说是展平多维的卷积图。check:http://pytorch.org/docs/master/tensors.html?highlight=view#torch.Tensor.view
        output = self.fc1(output)
        return output
    
    def forward(self, input1, input2, input3):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        output3 = self.forward_once(input3)
        return output1,output2,output3


In [ ]:
net = TripletNetwork()
print(net)

## Online Training!

In [ ]:
# criterion = F.triplet_margin_loss()   
# criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0001)
counter = []
loss_history = [] 
valid_num = []
iteration_number= 0
batch_apn_path = Config.batch_apn_path
batch_all_hard_path = Config.batch_all_hard_path
class_batch_size = 5
pic_batch_size = 5

alpha = 0.2

In [ ]:
for epoch in range(0,Config.train_number_epoch):

##筛选有效三元组（all or semi-hard）
    batch_all_hard = []
    MiniBatchAllTriplets(data_path = Config.training_dir,  
                         batch_path = batch_apn_path,  
                         num_class = class_batch_size,
                         num_pic = pic_batch_size)
    batch_apn_datasets = GetBatchTriplet(img_path='./',
                                         txt_path=batch_apn_path,
                                         data_transforms=transforms.Compose([transforms.Resize((100,100)),
                                                                             transforms.ToTensor()
                                                                            ])
                                         )
    gethard_apn_dataloader = DataLoader(batch_apn_datasets,
                                        shuffle=False,
                                        batch_size=1)
    for i, data in enumerate(gethard_apn_dataloader,0):
        img0, img1 , img2 = data
        img0, img1 , img2 = Variable(img0), Variable(img1) , Variable(img2)
        output1,output2,output3 = net(img0,img1,img2)
        ## Semi-hard:
        ###    [f(a)?f(p)]^2  <  [f(a)?f(n)]^2 
        eu_distance_ap = F.pairwise_distance(output1, output2)
        eu_distance_an = F.pairwise_distance(output1, output3)
        eu_dist_ap = eu_distance_ap.cpu().data.numpy()[0][0]
        eu_dist_an = eu_distance_an.cpu().data.numpy()[0][0]
        eu_dist_ap_2 = eu_dist_ap**2
        eu_dist_an_2 = eu_dist_an**2
        if  (eu_dist_an_2 < eu_dist_ap_2 + alpha):      #batch-all
#         if (eu_dist_ap_2 < eu_dist_an_2 and eu_dist_an_2 < eu_dist_ap_2 + alpha):          ##batch-semi-hard
            linecache.updatecache(batch_apn_path)
            all_apn_path = linecache.getline(batch_apn_path,i+1)
            batch_all_hard.append(all_apn_path)
    
    
    if batch_all_hard != []:
        print("Valid Triplet: {} \n".format(len(batch_all_hard)))
        list.append(valid_num,len(batch_all_hard))
        fileObject = open(batch_all_hard_path, 'w')
        for all_apn_path in batch_all_hard:
            fileObject.write(all_apn_path)  
    #       fileObject.write('\n')  
        fileObject.close()

    
    #### Training!
        batch_all_hard_datasets = GetBatchTriplet(img_path='./',
                                                  txt_path= batch_all_hard_path,
                                                  data_transforms=transforms.Compose([transforms.Resize((100,100)),
                                                                                      transforms.ToTensor()
                                                                                 ])
                                              )

        all_hard_dataloaders = DataLoader(batch_all_hard_datasets,
                                           batch_size=32,
                                           num_workers=4,
                                           shuffle=True)

        for i, data in enumerate(all_hard_dataloaders,0):
            img0, img1 , img2 = data
            img0, img1 , img2 = Variable(img0), Variable(img1) , Variable(img2)
            output1,output2,output3 = net(img0,img1,img2)
            optimizer.zero_grad()
            loss_triplet = F.triplet_margin_loss(output1,output2,output3,margin=alpha)
            loss_triplet.backward()
            optimizer.step()
            if i %10 == 0 :
                print("Epoch: {} | Current loss {}\n".format(epoch,loss_triplet.data[0]))
                iteration_number +=10
                counter.append(iteration_number)
                loss_history.append(loss_triplet.data[0])
        
    else:
        print("Epoch:{}. Got no valid triplet in this batch".format(epoch))
     
    torch.save(net.state_dict(),'tri_batch_all_params.pkl')
    
show_plot(counter,loss_history)
        
        
        

In [ ]:
plt.plot(valid_num)
plt.xlabel('epochs')
plt.ylabel('valid triplets')
plt.show()

In [ ]:
torch.save(net.state_dict(),'tri_batch_all_params.pkl')

In [ ]:
net.load_state_dict(torch.load('tri_batch_all_params.pkl'))

In [ ]:
class TripletNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)   #在self...imgs中返回随机项
        # 先取A-P对，保持循环，直到找到相同的类的图像
        while True:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)   #元组=（图片路径，类编号）
            if img0_tuple[1] == img1_tuple[1]:                         #图片编号相同则是同一个类（以文件夹区分）
                break
        # 再取A-N
        while True:
            img2_tuple = random.choice(self.imageFolderDataset.imgs)
            if img0_tuple[1] != img2_tuple[1]:
                break

        img0 = Image.open(img0_tuple[0])    #读取实际图像，tuple[0]即路径
        img1 = Image.open(img1_tuple[0])
        img2 = Image.open(img2_tuple[0])
        img0 = img0.convert("L")            #from PIL，把img转换为256级灰度图像， L：8-bit pixels,black and white
        img1 = img1.convert("L")
        img2 = img2.convert("L")

        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)    #将输入图像转换为反色图像
            img1 = PIL.ImageOps.invert(img1)
            img2 = PIL.ImageOps.invert(img2)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            
        return img0, img1, img2
            #返回 img0,img1,img2，完成A-P-N组合
    
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)          #数据集大小

In [ ]:
class TripletNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)   #在self...imgs中返回随机项
        # 先取A-P对，保持循环，直到找到相同的类的图像
        while True:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)   #元组=（图片路径，类编号）
            if img0_tuple[1] == img1_tuple[1]:                         #图片编号相同则是同一个类（以文件夹区分）
                break
        # 再取A-N
        while True:
            img2_tuple = random.choice(self.imageFolderDataset.imgs)
            if img0_tuple[1] != img2_tuple[1]:
                break

        img0 = Image.open(img0_tuple[0])    #读取实际图像，tuple[0]即路径
        img1 = Image.open(img1_tuple[0])
        img2 = Image.open(img2_tuple[0])
        img0 = img0.convert("L")            #from PIL，把img转换为256级灰度图像， L：8-bit pixels,black and white
        img1 = img1.convert("L")
        img2 = img2.convert("L")

        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)    #将输入图像转换为反色图像
            img1 = PIL.ImageOps.invert(img1)
            img2 = PIL.ImageOps.invert(img2)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            
        return img0, img1, img2
            #返回 img0,img1,img2，完成A-P-N组合
    
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)          #数据集大小

In [ ]:
folder_dataset_train = dset.ImageFolder(root=Config.training_dir)
triplet_dataset = TripletNetworkDataset(imageFolderDataset=folder_dataset_train,
                                       transform=transforms.Compose([transforms.Resize((100,100)),
                                                                    transforms.ToTensor()
                                                                    ]),
                                       should_invert=False)

train_distri_dataloader = DataLoader(triplet_dataset, 
                             num_workers=6, 
                             batch_size=1, 
                             shuffle=True)



same_one_train = np.array([])
diff_one_train = np.array([])


for i, data in enumerate(train_distri_dataloader, 0):   #将一个可遍历的数据对象组合为一个索引序列，同时列出数据和数据下标
    img0, img1, img3 = data
    img0, img1, img3 = Variable(img0), Variable(img1), Variable(img3)
    output1, output2, output3 = net(img0, img1,img3)   # network output
    eu_distance_same = F.pairwise_distance(output1, output2)
    eu_dist_same = eu_distance_same.cpu().data.numpy()[0][0]
    eu_distance_diff = F.pairwise_distance(output1, output3)
    eu_dist_diff = eu_distance_diff.cpu().data.numpy()[0][0]
    same_one_train = np.append(same_one_train, eu_dist_same)
    diff_one_train = np.append(diff_one_train, eu_dist_diff) 

In [ ]:
sns.distplot(same_one_train, 
             #bins=20, 
             rug=False, 
             hist=True,
            label='Samples from same classes')
sns.distplot(diff_one_train, 
             #bins=20, 
             rug=False, 
             hist=True,
            label='Samples from different classes')
plt.xlabel('Dissimilarity(Euclidean Distance)')
plt.ylabel('Relative quantity')
plt.legend()
plt.show()

In [ ]:
folder_dataset_valida = dset.ImageFolder(root=Config.validation_dir)
triplet_dataset = TripletNetworkDataset(imageFolderDataset=folder_dataset_valida,
                                       transform=transforms.Compose([transforms.Resize((100,100)),
                                                                    transforms.ToTensor()
                                                                    ]),
                                       should_invert=False)

valida_distri_dataloader = DataLoader(triplet_dataset, 
                             num_workers=6, 
                             batch_size=1, 
                             shuffle=True)



same_one_valida = np.array([])
diff_one_valida= np.array([])


for i, data in enumerate(valida_distri_dataloader, 0):   #将一个可遍历的数据对象组合为一个索引序列，同时列出数据和数据下标
    img0, img1, img3 = data
    img0, img1, img3 = Variable(img0), Variable(img1), Variable(img3)
    output1, output2, output3 = net(img0, img1,img3)   # network output
    eu_distance_same = F.pairwise_distance(output1, output2)
    eu_dist_same = eu_distance_same.cpu().data.numpy()[0][0]
    eu_distance_diff = F.pairwise_distance(output1, output3)
    eu_dist_diff = eu_distance_diff.cpu().data.numpy()[0][0]
    same_one_valida = np.append(same_one_valida, eu_dist_same)
    diff_one_valida = np.append(diff_one_valida, eu_dist_diff)

In [ ]:
sns.distplot(same_one_valida, 
             #bins=20, 
             rug=False, 
             hist=True,
            label='Samples from same classes')
sns.distplot(diff_one_valida, 
             #bins=20, 
             rug=False, 
             hist=True,
            label='Samples from different classes')
plt.xlabel('Dissimilarity(Euclidean Distance)')
plt.ylabel('Relative quantity')
plt.legend()
plt.show()

In [ ]:
tpr_array = np.array([])
fpr_array = np.array([])
same_one_valida = np.array([])
diff_one_valida = np.array([])


for thres in range(0,600,1):
    print(thres)
    tpm = 0
    fpm = 0
    tp_fn = 0
    fp_tn = 0
    for i, data in enumerate(valida_distri_dataloader, 0):
        img0, img1, img2 = data
        img0, img1, img2 = Variable(img0), Variable(img1), Variable(img2)
        output1, output2, output3 = net(img0, img1, img2)   # network output
        eu_distance_ap = F.pairwise_distance(output1, output2)
        eu_dist_ap = eu_distance_ap.cpu().data.numpy()[0][0]
        eu_distance_an = F.pairwise_distance(output1, output3)
        eu_dist_an = eu_distance_an.cpu().data.numpy()[0][0]
        
        tp_fn = tp_fn + 1
        fp_tn = fp_tn + 1
        
        if eu_dist_ap < (thres/100):
            tpm = tpm + 1
            
        if eu_dist_an < (thres/100):
            fpm = fpm + 1

    try:
        tpr=tpm/tp_fn
    except:
        print("still 0")
    else:
        tpr_array = np.append(tpr_array,tpr)
    try:
        fpr=fpm/fp_tn
    except:
        print("still 0")
    else:
        fpr_array = np.append(fpr_array,fpr)


In [ ]:
print(tp_fn,fp_tn,tpm,fpm)
len(fpr_array)
x = fpr_array
y = tpr_array 
sia_adam_roc = (x,y)
sia_adam_roc

In [ ]:
x = sia_adam_roc[0]
y = sia_adam_roc[1]

# This is the ROC curve
plt.plot(x,y,'*')
plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# This is the AUC
auc = np.trapz(y,x)
print(auc)